In [ ]:
import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import hist
from hist import Hist

import time

import myPIDselector

import math

import babar_analysis_tools as bat

# PID selectors and masks

For reference

https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/PID/PID_selectors_table


## Calculate the bits for the selector map for tracks

There are two functions we'll make use of in `babar_analysis_tools`

### `calculate_bits_for_PID_selector`

This function takes two or three inputs: 
* `trkidx` which is the index of the (charged) tracks to which a particle (e.g. proton) points back to
* `trk_selector_map` which is the bit-packed representation of the selectors for each track for a given particle hypothesis (proton, pion, etc.)
* `verbose` if you want more output set this to 1

It returns an integeer that represents the binary representation of the selectors as an `awkward` array. For example, `11001` or `1000001` but there are no leading zeros.


### `mask_PID_selection`

This function takes three inputs
* `bits` the binary representation of the selectors for a given track. This is the output of `calculate_bits_for_PID_selector`.
* `selector` is the particular selector you want to see if the tracks passed (e.g. `TightKMProtonSelection`)
* `pid_map_object` is the `PIDselector` class written by Bellis and located in `myPIDselector.py`. It contains information about what selectors are available for each particle hypothesis and what bit locations they map on to.

## Examples of usage

First grab a parquet file.


In [ ]:
# Parquet

# This is what we have at Siena
topdir = '/mnt/qnap/babar_data/bnv_plambda'

# Background
filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'

# Signal
#filename = f'{topdir}/Signal_SP_mode.parquet'

start = time.time()

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")

print(type(data))

### Using `calculate_bits_for_PID_selector`

Get the bits for the proton hypothesis for all the protons in our event.

Note that the number of protons is probably less than (definitely not more than!) the number of tracks. 

The number of protons just comes from our decay chain hypothesis. 

In [ ]:
print("protons ====================")

# Get the track index for the protons 
# For everything we call a proton, this tells you where in the track list you would 
# find that charged track
trkidx = data['pTrkIdx']
print(f"trkidx: \n{trkidx}\n")

# We can get the selector map for all tracks for protons (for example)
trk_selector_map = data['pSelectorsMap']
print(f"trk_selector_map:\n{trk_selector_map}\n")

# Unpack the bits 
print("Unpack the bits...")
# Note that we are running the function with verbose=1 so there is some output from the function
pbits = bat.calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=1)
print(f"pbits\n{pbits}\n")

print("Double check that the trk selector map and the returned `pbits` both have the same number of entries")
print(len(trk_selector_map))
print(len(pbits))
print()

## Using `mask_PID_selection`

We'll use this to create a mask for events based on whether or not the the protons pass some selector.

In [ ]:
# We will use this
pps = myPIDselector.PIDselector("p")

pps.selectors

In [ ]:
# Comment and uncomment the following to try different selectors.
#selector = 'LooseKMProtonSelection'
#selector = 'TightKMProtonSelection'
selector = 'VeryLooseLHProtonSelection'

print(f"What protons pass the {selector} threshold")

# We explained this part in the previous cell
trkidx = data['pTrkIdx']
print(f"trkidx: \n{trkidx}\n")

# We can get the selector map for all tracks for protons (for example)
trk_selector_map = data['pSelectorsMap']
print(f"trk_selector_map:\n{trk_selector_map}\n")

# Make use of the `calculate_bits_for_PID_selector` function, same as previous cell
# but we'll set the verbosity to 0
pbits = bat.calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=0)
print(f"pbits\n{pbits}\n")

# Now we will use this to make a mask. First we need to get the selector information
# from our homegrown class.
pps = myPIDselector.PIDselector("p")

# Use this and the bits we calculated 
mask_pid = bat.mask_PID_selection(pbits, selector, pps)
print(f"mask_pid\n{mask_pid}\n")

print()
print("Before the mask")
print(data['penergy'])
print(ak.num(data['penergy']))
print(ak.num(data['penergy'], axis=0))
print(ak.sum(ak.num(data['penergy'])))
print()

print("After the mask")
print(data['penergy'][mask_pid])
print(ak.num(data['penergy'][mask_pid]))
print(ak.num(data['penergy'][mask_pid], axis=0))
print(ak.sum(ak.num(data['penergy'][mask_pid])))
print()

## Masking decay products

Now let's use this to mask decay products of our Lambda.

Each Lambda has two decay products: a proton and a pion. 

We access the index of the decay products using `Lambda0d1Idx` (or `Lambda0d2Idx`).

We access the Lund ID of the decay products using `Lambda0d1Lund` (or `Lambda0d2Lund`). ([reference for Lund ID scheme](https://pdg.lbl.gov/2007/reviews/montecarlorpp.pdf) )

We can then use this as we did in the previous cells. 

In [ ]:
# Get the index of the decay products and the Lund IDs to see what we have. 
d1idx = data['Lambda0d1Idx']#[:,0]
d2idx = data['Lambda0d2Idx']#[:,0]
d1lund = data['Lambda0d1Lund']#[:,0]
d2lund = data['Lambda0d2Lund']#[:,0]

print("Daughter 1...")
print("d1idx")
print(d1idx)
print("d1lund")
print(d1lund)
print()

print("Daughter 2...")
print("d2idx")
print(d2idx)
print("d2lund")
print(d2lund)
print()

# It looks like the first daughter is always the proton and the second daughter
# is always the pion. 
print("\nGet the bits for the protons...")

# We explained this part in the previous cell
trkidx = data['pTrkIdx']
print(f"trkidx: \n{trkidx}\n")

# We can get the selector map for all tracks for protons (for example)
trk_selector_map = data['pSelectorsMap']
print(f"trk_selector_map:\n{trk_selector_map}\n")

# Calculate the binary representation
pbits = bat.calculate_bits_for_PID_selector(trkidx, trk_selector_map, verbose=0)
print(pbits)
print(pbits[d1idx])
print()

selector = 'LooseKMProtonSelection'
print(f"Now trying to create a mask with {selector}")

# Now we will use this to make a mask. First we need to get the selector information
# from our homegrown class.
pps = myPIDselector.PIDselector("p") # This is a helpful toolbox!

# Use this and the bits we calculated 
mask_pid = bat.mask_PID_selection(pbits, selector, pps)
print(f"mask_pid\n{mask_pid}\n")
print()

# Need to use these for the protons that came from the Lambda
passing_pbits = pbits[d1idx][mask_pid]
print("\nGet the masked bits of the first daughter of the Lambda...")
print(pbits[d1idx][mask_pid])
print()


# Use the mask for the Lambda0 decay products 
mass = data['Lambda0_unc_Mass']

print("Lambda0_unc_Mass")
print(mass)
print()

# Make some plots!

plt.figure(figsize=(12,5));
plt.subplot(1,2,1)
plt.hist(ak.flatten(mass),bins=100)
plt.hist(ak.flatten(mass[mask_pid]),bins=100)

print(len(ak.flatten(mass)))
print(len(ak.flatten(mass[mask_pid])))
print()

plt.subplot(1,2,2)

# Weirdness

print("Try this for the B meson")

Bmes = data['BpostFitMes']#[:,0]
FL = data['Lambda0FlightLen']#[:,0]
print(FL)
print(Bmes)

# For now, we do this because of weird second Lambda
#mask_fl = (data['Lambda0FlightLen']>=0)

#mask = mask_pid

plt.hist(ak.flatten(Bmes),bins=100, range=(5,5.3))
plt.hist(ak.flatten(Bmes[mask_pid]),bins=100, range=(5,5.3))

# Or maybe just this?
#plt.hist(Bmes[:,0],bins=100, range=(5,5.3))
#plt.hist(Bmes[mask_][:,0],bins=100, range=(5,5.3))


print(len(ak.flatten(Bmes)))
print(len(ak.flatten(Bmes[mask_pid])))

;

In [ ]:
#data.fields